In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
import torch
import h5py
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

class ParticleDataset(Dataset):
    def __init__(self, data_files, transform=None):
        self.data_files = data_files
        self.transform = transform
        self.data = []
        self.targets = []
        self.load_data()

    def load_data(self):
        for file_path in self.data_files:
            with h5py.File(file_path, "r") as f:
                dataset = f["X"][:]  
                labels = f["y"][:]   
                
                self.data.extend(dataset)
                self.targets.extend(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        target = int(self.targets[idx])  
    
        sample = sample.unsqueeze(0) if len(sample.shape) == 2 else sample  
    
        if self.transform:
            sample = self.transform(sample)
    
        return sample, torch.tensor(target, dtype=torch.long)  



In [ ]:
data_files = ["./dataset/photons.hdf5", "./dataset/electron.hdf5"]  


In [ ]:
dataset = ParticleDataset(data_files, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"Dataset Size: {len(dataset)}")
print(f"Training Samples: {len(train_dataset)}, Validation Samples: {len(val_dataset)}")

Dataset Size: 498000
Training Samples: 398400, Validation Samples: 99600


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  
        return torch.relu(out)

# Define the ResNet15 model
class ResNet15(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet15, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = ResNetBlock(64, 64)
        self.layer2 = ResNetBlock(64, 128, stride=2)
        self.layer3 = ResNetBlock(128, 256, stride=2)
        self.layer4 = ResNetBlock(256, 512, stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.initial(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

# Load Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize Model, Loss, Optimizer
model = ResNet15(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(train_loader):.4f} | Accuracy: {train_acc:.2f}%")

# Save the Model
torch.save(model.state_dict(), "resnet15_cifar10.pth")
print("Model saved successfully!")


100%|██████████| 170M/170M [00:37<00:00, 4.55MB/s] 


Epoch [1/10] | Loss: 1.2387 | Accuracy: 55.08%
Epoch [2/10] | Loss: 0.7746 | Accuracy: 72.66%
Epoch [3/10] | Loss: 0.5827 | Accuracy: 79.56%
Epoch [4/10] | Loss: 0.4379 | Accuracy: 84.73%
Epoch [5/10] | Loss: 0.3209 | Accuracy: 88.78%
Epoch [6/10] | Loss: 0.2209 | Accuracy: 92.21%
Epoch [7/10] | Loss: 0.1447 | Accuracy: 94.91%
Epoch [8/10] | Loss: 0.1070 | Accuracy: 96.32%
Epoch [9/10] | Loss: 0.0842 | Accuracy: 97.07%
Epoch [10/10] | Loss: 0.0656 | Accuracy: 97.72%
Model saved successfully!
